In [ ]:
import pandas as pd
import json
from typing import Dict, List
from collections import Counter

characters = '0123456789abcdefghijklmnopqrstuvwxyz'


def sum_object(obj):
    sum = 0
    for k, v in obj.items():
        sum += v
    return sum


def truncate_object(obj, n=None):
    if not n:
        return obj

    res = {}
    for k, v in obj.items():
        res[k] = v
        n -= 1
        if n == 0:
            break
    return res


def analysis_error_frequency(csv_path, freq=None):
    error_char: Dict[str, Dict[str, int]] = {}
    df = pd.read_csv(csv_path)
    for i in range(len(df)):
        predict = str(df.loc[i, "Prediction"])
        truth = str(df.loc[i, "Ground Truth"])
        if predict == truth or len(predict) != len(truth):
            continue
        for j in range(len(predict)):
            predict_char = predict[j]
            truth_char = truth[j]
            if predict_char == truth_char:
                continue
            if truth_char not in error_char:
                error_char[truth_char] = {
                    predict_char: 1
                }
            else:
                if predict_char not in error_char[truth_char]:
                    error_char[truth_char][predict_char] = 1
                else:
                    error_char[truth_char][predict_char] += 1

    # 对嵌套的字典进行排序
    for k, v in error_char.items():
        if freq:
            # 过滤低于 freq 的字符
            error_char[k] = {
                k1: v1 for k1, v1 in v.items() if v1 >= freq
            }

        error_char[k] = dict(
            sorted(error_char[k].items(), key=lambda item: item[1], reverse=True))

    # 删除空字典
    error_char = {k: v for k, v in error_char.items() if v}

    error_char = dict(
        sorted(error_char.items(), key=lambda item: sum_object(item[1]), reverse=True))

    return error_char


filepath_list = [
    './csv/wandb_export_2024-11-28T14_58_16.473+08_00.csv'
]
for filepath in filepath_list:
    res = analysis_error_frequency(filepath)
    print(json.dumps(truncate_object(res), ensure_ascii=False, indent=4))
    all_chars: Dict[str, int] = Counter()
    for char, char_dict in res.items():
        all_chars[char] += sum_object(char_dict) // len(char_dict)
        for char2, count2 in char_dict.items():
            all_chars[char2] += count2
    # 降序排序
    all_chars = dict(
        sorted(all_chars.items(), key=lambda item: item[1], reverse=True))

    characters = []
    freqency = []
    for char, count in all_chars.items():
        characters.append(char)
        freqency.append(count)
    print(characters)
    print(freqency)
    with open('./result/characters.json', 'w') as f:
        json.dump({
            "characters": ''.join(characters),
            "freqency": freqency
        }, f, ensure_ascii=False, indent=4)

['l', 'u', 'w', 'i', 'q', 'h', 'a', '1', 'd', 'r', '3', 'j', '7', 'v', 'n', '8', 'x', 'k', 'e', 'o', 'y', 't', '4', 'z', '0', '9', 'c', 's', 'b', 'g', 'f', '2', 'm', 'p', '6', '5']
[95, 58, 55, 45, 44, 36, 35, 32, 30, 29, 29, 28, 28, 28, 27, 25, 23, 22, 21, 19, 19, 17, 17, 16, 16, 15, 15, 15, 14, 14, 13, 13, 13, 12, 10, 8]
